In [1]:
!pip -q install --no-deps pyctcdecode kenlm pygtrie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"

In [3]:
import json
from pathlib import Path

import torch
import librosa

from transformers import AutoProcessor, AutoModelForCTC, pipeline

EN_MODEL = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
PHONE_MODEL = "jimregan/wav2vec2-xls-r-300m-phoneme-timit"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUT = Path("/kaggle/working")

pipe_en = pipeline("automatic-speech-recognition", model=EN_MODEL, device=0 if DEVICE=="cuda" else -1)
pipe_phone = pipeline("automatic-speech-recognition", model=PHONE_MODEL, device=0 if DEVICE=="cuda" else -1)

en_proc = AutoProcessor.from_pretrained(EN_MODEL)
en_model = AutoModelForCTC.from_pretrained(EN_MODEL).to(DEVICE).eval()

V2 = Path("/kaggle/input/download-dubliners/v2")

DONE = [
    "dubliners_06_joyce_64kb",
    "dubliners_08_joyce_64kb",
    "dubliners_10_joyce_64kb",
    "dubliners_12_joyce_64kb",
    "dubliners_13_joyce_64kb",
    "dubliners_14_joyce_64kb",
]

for base in (V2):
    prefix = base.name

    for file in base.rglob("*.mp3"):
        if file.stem in DONE:
            continue
        stem = f"{prefix}_{file.stem}"

        en_out = pipe_en(str(file), chunk_length_s=10, return_timestamps="word")
        phone_out = pipe_phone(str(file), chunk_length_s=10, return_timestamps="word")

        (OUT / f"{stem}_en.json").write_text(json.dumps(en_out, ensure_ascii=False), encoding="utf-8")
        (OUT / f"{stem}_phone.json").write_text(json.dumps(phone_out, ensure_ascii=False), encoding="utf-8")


2026-01-11 13:59:48.428648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768139988.672956      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768139988.742675      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768139989.314859      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768139989.314915      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768139989.314919      17 computation_placer.cc:177] computation placer alr

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Device set to use cpu


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

TypeError: 'PosixPath' object is not iterable